# Whisper large-v3 ファインチューニング Notebook
この Notebook は Hugging Face 公式ブログ (https://huggingface.co/blog/fine-tune-whisper) の流れを参考にしつつ、`openai/whisper-large-v3` を日本語 (必要に応じてロシア語へ変更可) 向けに学習・評価するためのテンプレートです。

以下の 3 段階で進めます:
1. データ準備と前処理
2. 学習 (Trainer)
3. 評価・推論・モデル共有

セル 21 に「未確定パラメータ & 質問」をまとめているので、先にそこを確認しながら進めてください。

In [1]:
# 1. 環境セットアップ (初回のみ実行 / 再起動後に再実行する場合あり)
%pip install --upgrade pip
%pip install transformers accelerate datasets torchaudio jiwer evaluate soundfile librosa bitsandbytes --quiet
# %pip install optimum  # 追加最適化が必要な場合
print("Installed packages.")

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Installed packages.


In [2]:
# 2. GPU / メモリ資源確認
import torch, os, json, platform
print('Torch version:', torch.__version__)
if torch.cuda.is_available():
    print('GPU count:', torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f'  GPU[{i}]:', torch.cuda.get_device_name(i))
    # nvidia-smi (環境により表示されないこともある)
    try:
        import subprocess, textwrap
        smi = subprocess.check_output(['nvidia-smi', '--query-gpu=memory.total,memory.free,name', '--format=csv,noheader']).decode()
        print('nvidia-smi:\n', smi)
    except Exception as e:
        print('nvidia-smi 実行不可:', e)
else:
    print('CUDA 非利用 (CPU のみ)')

Torch version: 2.8.0+cu128
GPU count: 1
  GPU[0]: NVIDIA GeForce RTX 5080
nvidia-smi:
 16303 MiB, 15889 MiB, NVIDIA GeForce RTX 5080



In [3]:
# 3. 再現性と基本設定 (乱数シード等)
import random, numpy as np, torch
from dataclasses import dataclass
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
PROJECT_NAME = 'whisper-large-v3-finetune-ru'
# 言語はロシア語に固定（日本語ではなくロシア語向け学習）
LANG = 'ru'   # 必要なら 'ja', 'en' 等に変更可能
TASK = 'transcribe'  # 翻訳用途なら 'translate'
print({'SEED': SEED, 'LANG': LANG, 'TASK': TASK, 'PROJECT_NAME': PROJECT_NAME})

# ロシア語特有の注意:
# - ё と е の正規化方針を決める (多くは е に統一)
# - 句読点保持有無を決める（学習/評価一貫性）
# - 外来語 / 名前（カタカナ→キリル表記）統一ルール


{'SEED': 42, 'LANG': 'ru', 'TASK': 'transcribe', 'PROJECT_NAME': 'whisper-large-v3-finetune-ru'}


In [4]:
# 4. データ取得・定義 (例: Common Voice やローカル JSON)
from datasets import load_dataset, Audio
# 例: Common Voice 日本語 (利用規約&ライセンス要確認)
# dataset = load_dataset('mozilla-foundation/common_voice_17_0', 'ja', split='train+validation')
# ローカル JSONL: {"audio":"path.wav","text":"文字列"}
# dataset = load_dataset('json', data_files={'train':'data/train.jsonl'})['train']
print('データセットを読み込んだら、最初の数件を確認してください。')

c:\Users\mmyui\GoogleDrive\Program\Python\transcription_for_kapra\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


データセットを読み込んだら、最初の数件を確認してください。
